In [8]:
import math
import matplotlib.pyplot as plt
from ipywidgets import widgets, VBox
from IPython.display import display, clear_output
def integer_multiplication(a, b):
    def draw_tree(ax, text, x, y, dx, dy, depth=0):
        ax.text(x, y, text, fontsize=8, ha='center', va='center',
                bbox=dict(boxstyle="round,pad=0.3", facecolor="white", edgecolor="black"))
        if depth < max_depth:
            try:
                a, b = map(int, text.split(' x '))
                mid_a, mid_b = a // 2, b // 2
                left_text = f'{mid_a} x {mid_b}'
                right_text = f'{mid_a} x {b - mid_b}'
                ax.plot([x, x - dx], [y, y - dy], color='black')
                ax.plot([x, x + dx], [y, y - dy], color='black')
                draw_tree(ax, left_text, x - dx, y - dy, dx / 2, dy, depth + 1)
                draw_tree(ax, right_text, x + dx, y - dy, dx / 2, dy, depth + 1)
            except ValueError:
                pass
    fig, ax = plt.subplots(figsize=(10, 6))
    max_depth = 4
    draw_tree(ax, f'{a} x {b}', 0.5, 1.0, 0.2, 0.15, depth=0)
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis('off')
    plt.title('Multiplication Tree Diagram', fontsize=12)
    plt.show()

    return a * b

def euclidean_distance(p1, p2):
    return math.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)
def closest_in_strip(strip, d):
    min_dist = d
    strip.sort(key=lambda point: point[1])
    closest_pair = None
    for i in range(len(strip)):
        for j in range(i + 1, len(strip)):
            if (strip[j][1] - strip[i][1]) < min_dist:  # If the points are close enough in y-coordinate
                dist = euclidean_distance(strip[i], strip[j])
                if dist < min_dist:
                    min_dist = dist
                    closest_pair = (strip[i], strip[j])
    return closest_pair, min_dist


def closest_pair_of_points(points):
    def closest_util(points_x, points_y):
        # Base case: If there are 2 or 3 points, use brute force
        if len(points_x) <= 3:
            min_dist = float('inf')
            closest_pair = None
            for i in range(len(points_x)):
                for j in range(i + 1, len(points_x)):
                    dist = euclidean_distance(points_x[i], points_x[j])
                    if dist < min_dist:
                        min_dist = dist
                        closest_pair = (points_x[i], points_x[j])
            return closest_pair, min_dist

        mid = len(points_x) // 2
        left_x = points_x[:mid]
        right_x = points_x[mid:]
        mid_x = points_x[mid][0]

        # Divide points into left and right halves based on x-coordinate
        left_y = [point for point in points_y if point[0] <= mid_x]
        right_y = [point for point in points_y if point[0] > mid_x]

        # Recursively find the closest pair in each half
        left_pair, left_dist = closest_util(left_x, left_y)
        right_pair, right_dist = closest_util(right_x, right_y)

        # Initialize closest_pair and min_dist before comparison
        if left_dist < right_dist:
            min_dist = left_dist
            closest_pair = left_pair
        else:
            min_dist = right_dist
            closest_pair = right_pair

        # Check if there is a closer pair in the strip (points close to the division line)
        strip = [point for point in points_y if abs(point[0] - mid_x) < min_dist]
        strip_pair, strip_dist = closest_in_strip(strip, min_dist)

        # Return the closest pair and minimum distance found
        if strip_dist < min_dist:
            return strip_pair, strip_dist
        return closest_pair, min_dist

    # Sort points by x and y coordinates
    points_x = sorted(points, key=lambda point: point[0])
    points_y = sorted(points, key=lambda point: point[1])

    return closest_util(points_x, points_y)

algorithm_selector = widgets.Dropdown(
    options=['Closest Pair of Points', 'Integer Multiplication'],
    description='Algorithm:',
    style={'description_width': 'initial'},
)

file_uploader = widgets.FileUpload(
    accept='.txt',
    multiple=False,
    description="Upload File"
)

process_button = widgets.Button(
    description="Process",
    button_style='success',
    tooltip="Click to process the uploaded file"
)
output_area = widgets.Output()
def process_file(change):
    with output_area:
        clear_output()
        if not file_uploader.value:
            print("Please upload a file.")
            return
        uploaded_file = next(iter(file_uploader.value.values()))
        file_content = uploaded_file['content'].decode('utf-8')
        if algorithm_selector.value == 'Closest Pair of Points':
            try:
                points = [tuple(map(float, line.split())) for line in file_content.strip().split('\n')]
                result, distance = closest_pair_of_points(points)
                x_points = [point[0] for point in points]
                y_points = [point[1] for point in points]
                plt.scatter(x_points, y_points, color='blue', label="Points")

                p1, p2 = result
                plt.plot([p1[0], p2[0]], [p1[1], p2[1]], color='red', linewidth=2, label="Closest Pair")

                plt.text(p1[0], p1[1], f'({p1[0]:.2f}, {p1[1]:.2f})', fontsize=9)
                plt.text(p2[0], p2[1], f'({p2[0]:.2f}, {p2[1]:.2f})', fontsize=9)

                plt.title(f"Closest Pair: Distance = {distance:.4f}")
                plt.xlabel("X")
                plt.ylabel("Y")
                plt.legend()
                plt.show()

                print(f"\nClosest Pair of Points:")
                print(f"Points: {result[0]} and {result[1]}")
                print(f"Distance: {distance:.4f}")
            except Exception as e:
                print(f"Error: {e}")
        elif algorithm_selector.value == 'Integer Multiplication':
            try:
                a, b = map(int, file_content.strip().split())
                result = integer_multiplication(a, b)
                print(f"\nInteger Multiplication Result:")
                print(f"{a} x {b} = {result}")
            except Exception as e:
                print(f"Error: {e}")
process_button.on_click(process_file)
display(VBox([algorithm_selector, file_uploader, process_button, output_area]))


